Dependencies
---------------

In [1]:
!pip install gammatone
!pip install librosa
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 92.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 55.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 87.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.8/412.8 kB 73.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import librosa
import scipy.stats
from scipy.fftpack import dct
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
# CHANGED: correct UMAP import
import umap.umap_ as umap
import plotly.express as px
from scipy.signal import butter, lfilter
from scipy.optimize import linear_sum_assignment
from typing import Dict, Any, Tuple, Optional
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
from gammatone.filters import centre_freqs, make_erb_filters, erb_filterbank
from scipy.fftpack import dct

# NEW: internal clustering metrics
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)

Runner and report for PAM data
---------------------------

In [2]:
def gtcc_frames(frames, sr, n_bands=64, low_hz=50, high_hz=None, n_ceps=80):
    if high_hz is None:
        high_hz = sr / 2 - 1
    cf = centre_freqs(sr, n_bands, low_hz, high_hz)
    fb = make_erb_filters(sr, cf)
    energies = []
    for f in frames:
        y = erb_filterbank(f, fb)
        e = np.sum(y**2, axis=1) + 1e-10
        energies.append(e)
    E = np.vstack(energies)
    logE = np.log(E)
    C = dct(logE, type=2, axis=1, norm='ortho')[:, :n_ceps]
    return C

# ----------------------------
# Feature & utility functions
# ----------------------------
def bandpass_filter(signal, sr=16000, lowcut=0, highcut=2000, order=4):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, signal)

def compute_deltas(feat_matrix, N=2):
    num_frames = feat_matrix.shape[0]
    padded = np.pad(feat_matrix, ((N, N), (0, 0)), mode='edge')
    deltas = np.zeros_like(feat_matrix)
    denom = 2 * sum(i ** 2 for i in range(1, N + 1))
    for t in range(num_frames):
        for n in range(1, N + 1):
            deltas[t] += n * (padded[t + N + n] - padded[t + N - n])
        deltas[t] /= denom
    return deltas

def extract_gtcc_mfcc(audio, sr=16000, n_bands=64, n_coeffs=80):
    win_size = int(sr*0.025); hop = int(sr*0.010)
    if len(audio) < win_size:
        gtcc_mean = np.zeros(n_coeffs, dtype=np.float32)
    else:
        frames = librosa.util.frame(audio.astype(np.float32), frame_length=win_size, hop_length=hop).T
        window = np.hanning(win_size).astype(np.float32)
        frames = frames * window
        gtcc = gtcc_frames(frames, sr, n_bands=n_bands, n_ceps=n_coeffs)
        gtcc_mean = gtcc.mean(axis=0)

    mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=1024, hop_length=512, n_mels=64)
    logmel = librosa.power_to_db(mel)
    mfcc = librosa.feature.mfcc(S=logmel, sr=sr, n_mfcc=13).mean(axis=1)

    return np.concatenate([gtcc_mean, mfcc])

def _extract_one_feature(index_and_path):
    idx, path = index_and_path
    audio = np.load(path).astype(np.float32)
    maxabs = np.max(np.abs(audio))
    if maxabs > 0:
        audio = audio / maxabs
    feats = extract_gtcc_mfcc(audio)
    return idx, feats

def load_gtcc_mfcc_features(audio_dir, label_path):
    audio_paths = sorted(
        [os.path.join(audio_dir, f) for f in os.listdir(audio_dir)
         if f.endswith(".npy") and f != "labels.npy"],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )
    labels = np.load(label_path)

    X = [None] * len(audio_paths)
    y = []

    with ProcessPoolExecutor() as ex:
        futures = {
            ex.submit(_extract_one_feature, (i, p)): i
            for i, p in enumerate(audio_paths)
        }
        for fut in as_completed(futures):
            i = futures[fut]
            try:
                idx, feats = fut.result()
                X[idx] = feats
            except Exception as e:
                print(f"Skipping sample {i} ({audio_paths[i]}) due to error: {e}")
                X[i] = None

    X_out, y_out = [], []
    for i, feats in enumerate(X):
        if feats is not None:
            X_out.append(feats)
            y_out.append(labels[i])

    return np.asarray(X_out), np.asarray(y_out)

def hungarian_accuracy(y_true, y_pred):
    D = int(max(y_pred.max(), y_true.max())) + 1
    w = np.zeros((D, D), dtype=np.int64)
    np.add.at(w, (y_pred, y_true), 1)
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return float(sum(w[i, j] for i, j in zip(row_ind, col_ind)) / len(y_pred))

# NEW: helper to compute internal indices robustly
def _internal_indices(X: np.ndarray, labels: np.ndarray) -> Dict[str, float]:
    """Compute Silhouette, DB, CH with guards; returns NaN if invalid."""
    out = {"silhouette": float("nan"), "davies_bouldin": float("nan"), "calinski_harabasz": float("nan")}
    unique = np.unique(labels)
    if len(unique) <= 1 or len(unique) >= len(labels):
        return out
    try:
        out["silhouette"] = float(silhouette_score(X, labels))
    except Exception as _:
        pass
    try:
        out["davies_bouldin"] = float(davies_bouldin_score(X, labels))
    except Exception as _:
        pass
    try:
        out["calinski_harabasz"] = float(calinski_harabasz_score(X, labels))
    except Exception as _:
        pass
    return out

def run_kmeans_clustering(
    X_train, y_train, X_test, y_test,
    umap_html_path: str = "kmeans_testset_umap.html",
    *,
    # NEW: choose number of clusters; if None, infer from labels; set to 60 for fixed-k option
    kmeans_k: Optional[int] = None,
    # NEW: toggle computing internal indices on the test set
    compute_internal_indices: bool = True,
) -> Dict[str, float]:
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    n_clusters = int(kmeans_k) if kmeans_k is not None else len(np.unique(y_train))
    model = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    model.fit(X_train_scaled)

    y_pred = model.predict(X_test_scaled)

    ari = adjusted_rand_score(y_test, y_pred)
    ami = adjusted_mutual_info_score(y_test, y_pred)
    hacc = hungarian_accuracy(y_test, y_pred)

    print(f"\n📊 GTCC + MFCC Clustering Evaluation on Test Set (k={n_clusters})")
    print(f"ARI: {ari:.4f}")
    print(f"AMI: {ami:.4f}")
    print(f"Hungarian Accuracy: {hacc:.4f}")

    # NEW: internal indices on X_test
    extra = {}
    if compute_internal_indices:
        extra = _internal_indices(X_test_scaled, y_pred)
        print(f"Silhouette: {extra['silhouette']:.4f} | "
              f"Davies–Bouldin: {extra['davies_bouldin']:.4f} | "
              f"Calinski–Harabasz: {extra['calinski_harabasz']:.2f}")

    reducer = umap.UMAP(n_components=3, random_state=42)
    X_umap = reducer.fit_transform(X_test_scaled.astype(np.float32, copy=False))

    fig = px.scatter_3d(
        x=X_umap[:, 0], y=X_umap[:, 1], z=X_umap[:, 2],
        color=y_test.astype(str),
        title=f"KMeans (k={n_clusters}) | ARI {ari:.4f}, AMI {ami:.4f}, H-Acc {hacc:.4f}",
        labels={"x": "UMAP-1", "y": "UMAP-2", "z": "UMAP-3"},
        opacity=0.7
    )
    fig.update_traces(marker=dict(size=3))
    fig.update_layout(legend_title_text='True Label', width=800, height=700)
    fig.write_html(umap_html_path)
    print(f"Saved UMAP visualization to '{umap_html_path}'")

    out = {"ari": float(ari), "ami": float(ami), "hungarian_accuracy": float(hacc)}
    out.update(extra)
    return out

# ----------------------------
#  Runner
# ----------------------------
def run_pipeline(
    dataset_path: str,
    do_clustering: int = 1,
    *,
    test_size: float = 0.2,
    random_state: int = 42,
    stratify: bool = True,
    umap_html_path: str = "kmeans_testset_umap.html",
    # NEW: pass-through options
    kmeans_k: Optional[int] = None,
    compute_internal_indices: bool = True,
) -> Dict[str, Any]:
    """
    Load features from `dataset_path`, split train/test, and optionally run KMeans clustering.
    Expects a 'labels.npy' file inside `dataset_path` and per-utterance audio arrays as .npy files.

    NEW:
      - kmeans_k: set to 60 to force KMeans with 60 clusters; None infers from labels
      - compute_internal_indices: compute Silhouette, DB, CH on test set predictions
    """
    audio_dir = dataset_path
    label_path = os.path.join(audio_dir, "labels.npy")

    if not os.path.isdir(audio_dir):
        raise FileNotFoundError(f"Dataset directory not found: {audio_dir}")
    if not os.path.isfile(label_path):
        raise FileNotFoundError(f"Label file not found: {label_path}")

    X, y = load_gtcc_mfcc_features(audio_dir, label_path)
    if X.size == 0:
        raise ValueError("No features were loaded. Check your .npy files and indexing.")

    strat_arg = y if stratify else None
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=strat_arg
    )

    results: Dict[str, Any] = {
        "n_samples": int(len(X)),
        "n_features": int(X.shape[1]),
        "train_samples": int(len(X_train)),
        "test_samples": int(len(X_test)),
        "did_clustering": bool(do_clustering),
    }

    if do_clustering:
        metrics = run_kmeans_clustering(
            X_train, y_train, X_test, y_test,
            umap_html_path=umap_html_path,
            kmeans_k=kmeans_k,  # NEW
            compute_internal_indices=compute_internal_indices,  # NEW
        )
        results["metrics"] = metrics
        results["umap_html_path"] = umap_html_path

    results["splits"] = {
        "X_train": X_train, "y_train": y_train,
        "X_test": X_test, "y_test": y_test
    }
    return results

def run_pipeline_test_only(
    dataset_path: str,
    *,
    test_size: float = 0.02,
    random_state: int = 42,
    stratify: bool = True,
    umap_html_path: str = "kmeans_testset_umap.html",
    # NEW: separate option to force k=60
    kmeans_60: bool = False,
    # NEW: or specify any k directly; kmeans_60 takes precedence if True
    kmeans_k: Optional[int] = None,
    compute_internal_indices: bool = True,
) -> Dict[str, Any]:
    """
    Only process the test split (≈test_size of the dataset).
    Runs clustering directly on test set only.

    NEW:
      - kmeans_60=True forces KMeans with 60 clusters (separate option you asked for)
      - or set kmeans_k to any int; if both given, kmeans_60 wins
      - compute_internal_indices computes Silhouette, DB, CH
    """
    audio_dir = dataset_path
    label_path = os.path.join(audio_dir, "labels.npy")

    if not os.path.isdir(audio_dir):
        raise FileNotFoundError(f"Dataset directory not found: {audio_dir}")
    if not os.path.isfile(label_path):
        raise FileNotFoundError(f"Label file not found: {label_path}")

    audio_paths = sorted(
        [os.path.join(audio_dir, f) for f in os.listdir(audio_dir)
         if f.endswith(".npy") and f != "labels.npy"],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )
    labels = np.load(label_path)

    strat_arg = labels if stratify else None
    _, paths_test, _, y_test = train_test_split(
        audio_paths, labels, test_size=test_size,
        random_state=random_state, stratify=strat_arg
    )

    X_test = []
    for p in paths_test:
        audio = np.load(p).astype(np.float32)
        maxabs = np.max(np.abs(audio))
        if maxabs > 0:
            audio = audio / maxabs
        feats = extract_gtcc_mfcc(audio)
        X_test.append(feats)

    X_test = np.asarray(X_test)

    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)

    # NEW: choose k
    if kmeans_60:
        n_clusters = 60
    elif kmeans_k is not None:
        n_clusters = int(kmeans_k)
    else:
        n_clusters = len(np.unique(y_test))

    model = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    y_pred = model.fit_predict(X_test_scaled)

    ari = adjusted_rand_score(y_test, y_pred)
    ami = adjusted_mutual_info_score(y_test, y_pred)
    hacc = hungarian_accuracy(y_test, y_pred)

    print(f"\n📊 Test-Only KMeans Evaluation (k={n_clusters})")
    print(f"ARI: {ari:.4f}")
    print(f"AMI: {ami:.4f}")
    print(f"Hungarian Accuracy: {hacc:.4f}")

    # NEW: internal indices
    extra = {}
    if compute_internal_indices:
        extra = _internal_indices(X_test_scaled, y_pred)
        print(f"Silhouette: {extra['silhouette']:.4f} | "
              f"Davies–Bouldin: {extra['davies_bouldin']:.4f} | "
              f"Calinski–Harabasz: {extra['calinski_harabasz']:.2f}")

    reducer = umap.UMAP(n_components=3, random_state=42)
    X_umap = reducer.fit_transform(X_test_scaled.astype(np.float32, copy=False))

    fig = px.scatter_3d(
        x=X_umap[:, 0], y=X_umap[:, 1], z=X_umap[:, 2],
        color=y_test.astype(str),
        title=f"Test-Only KMeans (k={n_clusters}) | ARI {ari:.4f}, AMI {ami:.4f}, H-Acc {hacc:.4f}",
        labels={"x": "UMAP-1", "y": "UMAP-2", "z": "UMAP-3"},
        opacity=0.7
    )
    fig.update_traces(marker=dict(size=3))
    fig.update_layout(legend_title_text='True Label', width=800, height=700)
    fig.write_html(umap_html_path)
    print(f"Saved UMAP visualization to '{umap_html_path}'")

    metrics = {
        "ari": float(ari),
        "ami": float(ami),
        "hungarian_accuracy": float(hacc),
    }
    metrics.update(extra)

    return {
        "test_samples": int(len(X_test)),
        "n_features": int(X_test.shape[1]),
        "metrics": metrics,
        "umap_html_path": umap_html_path,
        "splits": {
            "X_test": X_test, "y_test": y_test
        }
    }

2025-08-27 15:31:41.067668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-27 15:31:41.067735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-27 15:31:41.151712: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-27 15:31:41.325625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-27 15:31:42.820421: W tensorflow/compiler/tf2

In [4]:
# ----------------------------
# Example usage
# ----------------------------
res = run_pipeline_test_only("/notebooks/dataset_preprocessed", kmeans_60=False, kmeans_k=6)
print(res["metrics"])  # includes silhouette, DB, CH
print("done")


📊 Test-Only KMeans Evaluation (k=6)
ARI: 0.0912
AMI: 0.1602
Hungarian Accuracy: 0.3380
Silhouette: 0.2317 | Davies–Bouldin: 1.4524 | Calinski–Harabasz: 1535.81


/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Saved UMAP visualization to 'kmeans_testset_umap.html'
{'ari': 0.09124351671746268, 'ami': 0.16023243740207158, 'hungarian_accuracy': 0.33801112992983307, 'silhouette': 0.23171464457016527, 'davies_bouldin': 1.45241012006733, 'calinski_harabasz': 1535.8141682878859}
done


In [2]:
!pip install soundata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.0/162.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_ext

Runner and report for urbansound8k dataset
---------------

In [3]:
# urbansound8k_metrics_mfcc_gtcc_AUTODOWNLOAD.py
# -----------------------------------------------------------
# TEST-only UrbanSound8K clustering with auto-download (soundata)
# - Downloads & validates UrbanSound8K (idempotent)
# - Extracts MFCC + GTCC from WAVs
# - KMeans clustering on TEST features only
# - UNSUPERVISED METRICS ONLY:
#     Silhouette (cosine), Davies–Bouldin, Calinski–Harabasz,
#     Intra-cluster cosine (mean)
# -----------------------------------------------------------

import os
import sys
import argparse
import numpy as np
import glob
import soundata

from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

import librosa
from gammatone.filters import centre_freqs, make_erb_filters, erb_filterbank
from scipy.fftpack import dct


# ---------- Feature extraction (GTCC + MFCC) ----------
def gtcc_frames(frames, sr, n_bands=64, low_hz=50, high_hz=None, n_ceps=80):
    if high_hz is None:
        high_hz = sr / 2 - 1
    cf = centre_freqs(sr, n_bands, low_hz, high_hz)
    fb = make_erb_filters(sr, cf)
    energies = []
    for f in frames:
        y = erb_filterbank(f, fb)
        e = np.sum(y ** 2, axis=1) + 1e-10
        energies.append(e)
    E = np.vstack(energies)
    logE = np.log(E)
    C = dct(logE, type=2, axis=1, norm="ortho")[:, :n_ceps]
    return C

def extract_gtcc_mfcc(audio, sr=32000, n_bands=64, n_ceps=80, n_mfcc=13):
    """Returns a single vector: [gtcc_mean (n_ceps) ; mfcc_mean (n_mfcc)]."""
    audio = np.asarray(audio, dtype=np.float32)
    if audio.size:
        maxabs = float(np.max(np.abs(audio)))
        if maxabs > 0:
            audio = audio / maxabs

    # 25 ms window / 10 ms hop @ 32 kHz
    win_size = int(sr * 0.025)
    hop = int(sr * 0.010)

    if len(audio) < max(2, win_size):
        gtcc_mean = np.zeros(n_ceps, dtype=np.float32)
    else:
        frames = librosa.util.frame(audio, frame_length=win_size, hop_length=hop).T
        frames = frames * np.hanning(win_size).astype(np.float32)
        gtcc = gtcc_frames(frames, sr, n_bands=n_bands, n_ceps=n_ceps)
        gtcc_mean = gtcc.mean(axis=0).astype(np.float32)

    mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=1024, hop_length=512, n_mels=64)
    logmel = librosa.power_to_db(mel, ref=np.max)
    mfcc = librosa.feature.mfcc(S=logmel, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = (mfcc.mean(axis=1) if mfcc.size else np.zeros(n_mfcc, dtype=np.float32)).astype(np.float32)

    return np.concatenate([gtcc_mean, mfcc_mean], dtype=np.float32)

def extract_from_wav(path, target_sr=32000):
    y, _ = librosa.load(path, sr=target_sr, mono=True)
    return extract_gtcc_mfcc(y, sr=target_sr)


# ---------- Utils ----------
def l2n(A):
    return A / (np.linalg.norm(A, axis=1, keepdims=True) + 1e-12)

def intra_cluster_cosine(X, labels):
    """Mean pairwise cosine similarity within clusters (excluding diagonal)."""
    Xn = l2n(X)
    per_cluster = []
    for c in np.unique(labels):
        idxs = np.where(labels == c)[0]
        if idxs.size < 2:
            continue
        S = cosine_similarity(Xn[idxs])
        m = (S.sum() - len(idxs)) / (len(idxs) * len(idxs) - 1)
        per_cluster.append(m)
    return (float(np.mean(per_cluster)) if per_cluster else 0.0), np.array(per_cluster)


# ---------- Main ----------
def main(args):
    os.makedirs(args.out_dir, exist_ok=True)

    print("Initializing UrbanSound8K dataset with soundata...")
    dataset = soundata.initialize("urbansound8k")

    # Download first so the dataset index exists (idempotent if already present)
    try:
        print("Ensuring dataset is present (this is idempotent)...")
        dataset.download()
    except Exception as e:
        print(f"[warn] soundata.download() raised: {e}")

    # Validate (optional info only)
    try:
        missing, invalid = dataset.validate()
        if missing or invalid:
            print(f"[warn] Validation issues — missing: {len(missing)}, invalid: {len(invalid)}")
    except FileNotFoundError:
        print("[warn] Could not load dataset index for validation. Continuing.")

    # Resolve dataset root (handle nested UrbanSound8K/)
    base = dataset.data_home
    # Prefer <data_home>/UrbanSound8K/audio if it exists; else <data_home>/audio
    audio_root = os.path.join(base, "UrbanSound8K", "audio")
    if not os.path.isdir(audio_root):
        audio_root = os.path.join(base, "audio")
    if not os.path.isdir(audio_root):
        raise SystemExit(
            f"Could not find audio directory under {base}. "
            "Check your soundata cache or extractor."
        )
    print(f"Using audio root: {audio_root}")

    # List WAVs (all folds)
    wavs = sorted(glob.glob(os.path.join(audio_root, "fold*", "*.wav")))
    if not wavs:
        # fallback: any wavs under audio_root
        wavs = sorted(glob.glob(os.path.join(audio_root, "**", "*.wav"), recursive=True))
    if not wavs:
        raise SystemExit("No WAV files found after download/validate.")

    # Optional random subset for speed
    MAX_TEST = 5000 if args.max_test is None else int(args.max_test)
    if MAX_TEST > 0 and len(wavs) > MAX_TEST:
        print(f"[subset] Reducing {len(wavs)} → {MAX_TEST}")
        rng = np.random.default_rng(42)
        sel = np.sort(rng.choice(len(wavs), size=MAX_TEST, replace=False))
        wavs = [wavs[i] for i in sel]

    print(f"Clips to process: {len(wavs)}")

    # Feature extraction
    feats = []
    print("[feat] Extracting features (GTCC+MFCC)...")
    for p in wavs:
        try:
            v = extract_from_wav(p, target_sr=args.sample_rate)
        except Exception as e:
            print(f"[warn] Failed on {p}: {e}")
            continue
        feats.append(v)

    if not feats:
        raise SystemExit("No features extracted (all files failed?).")

    X = np.asarray(feats, dtype=np.float32)
    print(f"Features: {X.shape} (dim={X.shape[1]})")

    # Standardize + L2 normalize
    Xs = StandardScaler().fit_transform(X).astype(np.float32, copy=False)
    Xn = l2n(Xs)

    # KMeans clustering
    km = KMeans(n_clusters=args.k_clusters, n_init=10, random_state=42)
    y_cluster = km.fit_predict(Xn)

    # Unsupervised metrics (guard for degenerate cases)
    def _safe(metric_fn, *a, **kw):
        try:
            return float(metric_fn(*a, **kw))
        except Exception as e:
            print(f"[warn] {metric_fn.__name__} not computed: {e}")
            return float("nan")

    sil = _safe(silhouette_score, Xn, y_cluster, metric="cosine")
    dbi = _safe(davies_bouldin_score, Xn, y_cluster)
    ch  = _safe(calinski_harabasz_score, Xn, y_cluster)
    cos_mean, per_cluster = intra_cluster_cosine(Xn, y_cluster)

    # Save report
    rep = os.path.join(args.out_dir, "report_test_only_unsup.txt")
    with open(rep, "w") as f:
        f.write("UrbanSound8K metrics (TEST-only; UNSUPERVISED ONLY)\n")
        f.write(f"Sample rate: {args.sample_rate}\n")
        f.write(f"KMeans k={args.k_clusters}\n")
        f.write(f"Silhouette (cosine): {sil:.6f}\n" if np.isfinite(sil) else "Silhouette (cosine): N/A\n")
        f.write(f"Davies–Bouldin:      {dbi:.6f}\n" if np.isfinite(dbi) else "Davies–Bouldin:      N/A\n")
        f.write(f"Calinski–Harabasz:   {ch:.6f}\n" if np.isfinite(ch) else "Calinski–Harabasz:   N/A\n")
        f.write(f"Intra-cluster cosine mean: {cos_mean:.6f}\n")
        f.write(f"Clusters >=2 items: {len(per_cluster)}\n")
    print("Saved metrics →", rep)


if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("--out_dir", type=str, default="./us8k_out", help="Output directory.")
    ap.add_argument("--sample_rate", type=int, default=32000)
    ap.add_argument("--k_clusters", type=int, default=10)
    ap.add_argument("--max_test", type=int, default=5000, help="Cap #clips; 0=use all.")
    args, unknown = ap.parse_known_args()
    if unknown:
        print("Ignoring unknown args:", unknown)
    main(args)


Ignoring unknown args: ['-f', '/root/.local/share/jupyter/runtime/kernel-859d66c2-3c5d-4f58-ba0f-3a3069499661.json']
Initializing UrbanSound8K dataset with soundata...


INFO: Downloading ['all', 'index']. Index is being stored in /usr/local/lib/python3.11/dist-packages/soundata/datasets/indexes, and the rest of files in /root/sound_datasets/urbansound8k
INFO: [all] downloading UrbanSound8K.tar.gz


Ensuring dataset is present (this is idempotent)...


5.61GB [06:15, 16.0MB/s]                                
INFO: [index] downloading urbansound8k_index_1.0.json
1.15MB [00:06, 197kB/s]                             
100%|██████████| 8732/8732 [00:13<00:00, 636.67it/s]
INFO: Success: the dataset is complete and all files are valid.
INFO: --------------------


[warn] Validation issues — missing: 2, invalid: 2
Using audio root: /root/sound_datasets/urbansound8k/audio
[subset] Reducing 8732 → 5000
Clips to process: 5000
[feat] Extracting features (GTCC+MFCC)...
Features: (5000, 77) (dim=77)
Saved metrics → ./us8k_out/report_test_only_unsup.txt


Runner and report for birdset dataset
----------------

In [2]:
# birdset_metrics_mfcc_gtcc_TESTONLY.py
# -----------------------------------------------------------
# Use prepared TEST audio (.npy) to:
#  - extract MFCC + GTCC features (no SimCLR)
#  - KMeans clustering (k=20) on TEST features only
#  - cluster metrics: silhouette, DBI, CH, intra-cluster cosine
#  - (optional, if test_labels.npy exists):
#       Hungarian hit@1 (cluster→class), NMI, ARI
#
# No train features or labels are touched. No labels regeneration.
#
# Dependencies: numpy, scipy, scikit-learn, librosa, gammatone
# -----------------------------------------------------------
import os
import argparse
import numpy as np
from glob import glob

from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    normalized_mutual_info_score,
    adjusted_rand_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import linear_sum_assignment

import librosa
from gammatone.filters import centre_freqs, make_erb_filters, erb_filterbank
from scipy.fftpack import dct


# ---------- Feature extraction (GTCC + MFCC) ----------
def gtcc_frames(frames, sr, n_bands=64, low_hz=50, high_hz=None, n_ceps=80):
    if high_hz is None:
        high_hz = sr / 2 - 1
    cf = centre_freqs(sr, n_bands, low_hz, high_hz)
    fb = make_erb_filters(sr, cf)
    energies = []
    for f in frames:
        y = erb_filterbank(f, fb)
        e = np.sum(y ** 2, axis=1) + 1e-10
        energies.append(e)
    E = np.vstack(energies)
    logE = np.log(E)
    C = dct(logE, type=2, axis=1, norm="ortho")[:, :n_ceps]
    return C

def extract_gtcc_mfcc(audio, sr=32000, n_bands=64, n_ceps=80, n_mfcc=13):
    """Returns a single vector: [gtcc_mean (n_ceps) ; mfcc_mean (n_mfcc)]."""
    audio = np.asarray(audio, dtype=np.float32)
    maxabs = float(np.max(np.abs(audio))) if audio.size else 0.0
    if maxabs > 0:
        audio = audio / maxabs

    # 25 ms window / 10 ms hop at 32 kHz
    win_size = int(sr * 0.025)
    hop = int(sr * 0.010)

    if len(audio) < max(2, win_size):
        gtcc_mean = np.zeros(n_ceps, dtype=np.float32)
    else:
        frames = librosa.util.frame(audio, frame_length=win_size, hop_length=hop).T
        window = np.hanning(win_size).astype(np.float32)
        frames = frames * window
        gtcc = gtcc_frames(frames, sr, n_bands=n_bands, n_ceps=n_ceps)
        gtcc_mean = gtcc.mean(axis=0).astype(np.float32)

    mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=1024, hop_length=512, n_mels=64)
    logmel = librosa.power_to_db(mel, ref=np.max)
    mfcc = librosa.feature.mfcc(S=logmel, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = (mfcc.mean(axis=1) if mfcc.size else np.zeros(n_mfcc, dtype=np.float32)).astype(np.float32)

    return np.concatenate([gtcc_mean, mfcc_mean], dtype=np.float32)

def _extract_one_feature(path, sr=32000):
    audio = np.load(path).astype(np.float32, copy=False)
    return extract_gtcc_mfcc(audio, sr=sr)


# ---------- Utils ----------
def l2n(A):
    return A / (np.linalg.norm(A, axis=1, keepdims=True) + 1e-12)

def intra_cluster_cosine(X, labels):
    Xn = l2n(X)
    per_cluster = []
    for c in np.unique(labels):
        idxs = np.where(labels == c)[0]
        if idxs.size < 2:
            continue
        S = cosine_similarity(Xn[idxs])
        m = (S.sum() - len(idxs)) / (len(idxs) * len(idxs) - 1)
        per_cluster.append(m)
    return (float(np.mean(per_cluster)) if per_cluster else 0.0), np.array(per_cluster)

def _list_npy_ordered(dirpath):
    paths = sorted(
        [p for p in glob(os.path.join(dirpath, "*.npy")) if os.path.basename(p) != "labels.npy"],
        key=lambda p: int(os.path.splitext(os.path.basename(p))[0])
    )
    return paths


# ---------- Main (TEST ONLY) ----------
def main(args):
    os.makedirs(args.out_dir, exist_ok=True)
    test_npy = os.path.join(args.out_dir, "test5s_npy")

    # Reuse prepared TEST .npy (collect all, then subset to cap)
    test_paths_all = _list_npy_ordered(test_npy)
    N_total = len(test_paths_all)
    if N_total == 0:
        raise SystemExit(
            f"Missing test npys. Found 0 in {test_npy}. "
            "Point --out_dir to your previous run with test5s_npy/."
        )

    # ---- Limit test set size (hard cap 5000, random but reproducible)
    MAX_TEST = 5000
    if N_total > MAX_TEST:
        print(f"[subset] Reducing test set: {N_total} → {MAX_TEST}")
        rng = np.random.default_rng(42)
        sel_idx = np.sort(rng.choice(N_total, size=MAX_TEST, replace=False))
        test_paths = [test_paths_all[i] for i in sel_idx]
    else:
        sel_idx = np.arange(N_total, dtype=int)
        test_paths = test_paths_all

    print(f"Reusing prepared TEST audio: {len(test_paths)} npy")

    # ---- Feature extraction (MFCC+GTCC) for TEST only
    print("[feat] Extracting TEST features (MFCC+GTCC)...")
    X_test = np.asarray([_extract_one_feature(p, sr=32000) for p in test_paths], dtype=np.float32)
    print(f"Features (test): {X_test.shape}  |  dim={X_test.shape[1]}")

    # ---- Standardize then L2-normalize (fit on TEST, since no train)
    scaler = StandardScaler()
    X_test_s = scaler.fit_transform(X_test).astype(np.float32, copy=False)
    X_test_n = l2n(X_test_s)

    # ---- KMeans clustering (on TEST)
    print(f"[cluster] KMeans(k={args.k_clusters}) on TEST features...")
    km = KMeans(n_clusters=args.k_clusters, n_init=10, random_state=42)
    y_cluster = km.fit_predict(X_test_n)

    # ---- Cluster metrics
    sil_cos = silhouette_score(X_test_n, y_cluster, metric="cosine")
    dbi     = davies_bouldin_score(X_test_n, y_cluster)
    ch      = calinski_harabasz_score(X_test_n, y_cluster)
    mean_cos, per_cluster = intra_cluster_cosine(X_test_n, y_cluster)

    # ---- Optional label-based metrics (requires test_labels.npy)
    labels_path = os.path.join(args.out_dir, "test_labels.npy")
    have_labels = os.path.isfile(labels_path)
    hungarian_hit1 = nmi = ari = None
    valid_map_mask = None
    N_labels_used = 0

    if have_labels:
        Y_test_all = np.load(labels_path)

        if Y_test_all.shape[0] == N_total:
            # Proper alignment with the selected subset
            Y_test = Y_test_all[sel_idx]
        else:
            # Fallback: warn and truncate to whatever we ended up using
            print("[warn] test_labels.npy length doesn't match all test npys; "
                  "falling back to naive truncation to match current subset.")
            Y_test = Y_test_all[:X_test.shape[0]]

        if Y_test.shape[0] != X_test.shape[0]:
            N = min(Y_test.shape[0], X_test.shape[0])
            print(f"[align] Truncating to {N} to match features/labels.")
            Y_test = Y_test[:N]
            X_test_n = X_test_n[:N]
            y_cluster = y_cluster[:N]

        # Classes present in TEST (ignore all-zero columns)
        classes = np.where(Y_test.sum(axis=0) > 0)[0]
        K = y_cluster.max() + 1
        C = len(classes)

        if C > 0 and K > 0:
            M = np.zeros((K, C), dtype=np.int64)
            for i in range(Y_test.shape[0]):
                pos = np.where(Y_test[i, classes] == 1)[0]
                if pos.size == 0:
                    continue
                M[y_cluster[i], pos] += 1

            sz = max(K, C)
            pad = np.zeros((sz, sz), dtype=np.int64)
            pad[:K, :C] = M
            cost = pad.max() - pad
            row_ind, col_ind = linear_sum_assignment(cost)
            cluster_to_class = {r: classes[c] for r, c in zip(row_ind, col_ind) if r < K and c < C}
            mapped_pred = np.array([cluster_to_class.get(k, -1) for k in y_cluster])
            valid_map_mask = (mapped_pred != -1)
            N_labels_used = int(valid_map_mask.sum())

            # hit@1 for multilabel: success if any true label == mapped class
            hits = (Y_test[np.arange(Y_test.shape[0]), np.clip(mapped_pred, 0, Y_test.shape[1]-1)] == 1)
            hungarian_hit1 = float(np.mean(hits[valid_map_mask])) if N_labels_used > 0 else None

            # NMI / ARI via single-label reduction (dominant test label)
            dom_gt = Y_test[:, classes].argmax(axis=1)
            # mask items with at least one positive among 'classes'
            valid_single = (Y_test[:, classes].sum(axis=1) > 0)
            if valid_single.any():
                nmi = normalized_mutual_info_score(dom_gt[valid_single], y_cluster[valid_single])
                ari = adjusted_rand_score(dom_gt[valid_single], y_cluster[valid_single])

    # ---- Save report
    report_path = os.path.join(args.out_dir, "report_test_only.txt")
    with open(report_path, "w") as f:
        f.write("Mode: TEST-ONLY\n")
        f.write(f"Test5s dir: {test_npy}\n")
        f.write(f"Features (test): {X_test.shape}\n")
        f.write(f"Feature dim: {X_test.shape[1]}\n")
        f.write(f"KMeans k={args.k_clusters}\n")
        f.write(f"Silhouette (cosine): {sil_cos:.6f}\n")
        f.write(f"Davies–Bouldin:      {dbi:.6f}\n")
        f.write(f"Calinski–Harabasz:   {ch:.6f}\n")
        f.write(f"Intra-cluster cosine mean: {mean_cos:.6f}\n")
        f.write(f"Clusters >=2 items: {len(per_cluster)}\n")
        f.write("\n")
        if have_labels:
            if hungarian_hit1 is not None:
                f.write(f"Hungarian hit@1 (cluster→class) [valid mapped items {N_labels_used}]: {hungarian_hit1:.6f}\n")
            else:
                f.write("Hungarian hit@1 (cluster→class): N/A\n")
            if nmi is not None:
                f.write(f"NMI (dominant test label, labeled N={int((Y_test.sum(axis=1)>0).sum())}): {nmi:.6f}\n")
            else:
                f.write("NMI: N/A\n")
            if ari is not None:
                f.write(f"ARI (dominant test label, labeled N={int((Y_test.sum(axis=1)>0).sum())}): {ari:.6f}\n")
            else:
                f.write("ARI: N/A\n")
        else:
            f.write("Label-based metrics: N/A (test_labels.npy not found)\n")

    print("Saved metrics ->", report_path)


if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("-f", "--fff", default=None, help="(ignored; Jupyter arg)")
    ap.add_argument("--out_dir", type=str, default="./birdset_bridge_out_32k",
                    help="Dir from your previous run (must contain test5s_npy/ and optionally test_labels.npy)")
    ap.add_argument("--k_clusters", type=int, default=20)
    args, unknown = ap.parse_known_args()
    if unknown:
        print("Ignoring unknown args:", unknown)
    main(args)

[subset] Reducing test set: 24480 → 5000
Reusing prepared TEST audio: 5000 npy
[feat] Extracting TEST features (MFCC+GTCC)...
Features (test): (5000, 77)  |  dim=77
[cluster] KMeans(k=20) on TEST features...
Saved metrics -> ./birdset_bridge_out_32k/report_test_only.txt
